In [7]:
from pyspark.sql import SparkSession

appName = "Employees Data discovery"
master = 'local[*]'

# Create Spark session
spark = SparkSession.builder \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

# Convert list to data frame
df = spark.read.format('csv') \
                .option('header',True) \
                .option('multiLine', True) \
                .load('../entities/employee/employees_100000.csv')

df.show()

+---------+----------+----------+----------------+--------------------+------------+----------+----------+-----+-------+--------------------+----------+
|    badge|first_name| last_name|      department|               phone|salary_cents|bonus_rate|  birthday|state|zipcode|               email|start_date|
+---------+----------+----------+----------------+--------------------+------------+----------+----------+-----+-------+--------------------+----------+
|985134917|    Sherry|    Martin|           Sales|        542.447.9638|    21029481|      0.38|1957-04-01|   SD|  57489|sherry.martin@com...|2018-07-28|
|522478832|  Kristina|      Diaz|Customer Support|        152.017.8523|    10119959|      1.05|1961-05-27|   GA|  30602|kristina.diaz@com...|2002-10-09|
|489400447| Alexandra|    Bishop|     Procurement| (971)469-9186x36059|    20856174|      0.61|1998-10-22|   OK|  73109|alexandra.bishop@...|2021-07-23|
|393500714|    Rachel|   Ballard|       Marketing| (859)263-6267x64571|     430563

In [11]:
from pyspark.sql.types import DecimalType
from operator import add
from pyspark.sql.functions import lit, col, when, floor, datediff, current_date, to_date
from pyspark.sql import functions as F

df = df.withColumn('calculated_age',floor(datediff(current_date(), to_date(col('birthday'), 'yyyy-M-d'))/365.25))
df = df.withColumn('calculated_job_age', floor(F.months_between(current_date(), to_date(col('start_date'), 'yyyy-M-d'))/12))
df = df.withColumn("calculated_total_compensation", (col("salary_cents")*(1+col('bonus_rate'))).cast(DecimalType(14,0)))
df.show()

+---------+----------+----------+----------------+--------------------+------------+----------+----------+-----+-------+--------------------+----------+--------------+------------------+-----------------------------+
|    badge|first_name| last_name|      department|               phone|salary_cents|bonus_rate|  birthday|state|zipcode|               email|start_date|calculated_age|calculated_job_age|calculated_total_compensation|
+---------+----------+----------+----------------+--------------------+------------+----------+----------+-----+-------+--------------------+----------+--------------+------------------+-----------------------------+
|985134917|    Sherry|    Martin|           Sales|        542.447.9638|    21029481|      0.38|1957-04-01|   SD|  57489|sherry.martin@com...|2018-07-28|            65|                 3|                     29020684|
|522478832|  Kristina|      Diaz|Customer Support|        152.017.8523|    10119959|      1.05|1961-05-27|   GA|  30602|kristina.dia

In [17]:

# from pyspark import StorageLevel
#
# df.groupBy("state").agg(F.avg('calculated_total_compensation'), F.count('state'), F.avg('calculated_job_age'), F.max('calculated_total_compensation')).show()
# df.createOrReplaceTempView('employees')
# df = df.persist(storageLevel=StorageLevel.DISK_ONLY)
q = "SELECT count(*) from employees"
spark.sql(q).show()

+--------+
|count(1)|
+--------+
|  100000|
+--------+

